In [2]:
import langchain
langchain.__version__ 

'0.3.15'

In [3]:
from langchain.agents import initialize_agent, Tool
from langchain.llms import OpenAI
from dotenv import load_dotenv
import os

In [6]:
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

In [8]:
from langchain_community.tools.ddg_search.tool import DuckDuckGoSearchRun

search_tool = Tool(
    name = "DuckDuckGo Search",
    func= DuckDuckGoSearchRun().run,
    description= "Use this tool to perform web searches using DuckDuckGo"
)

llm = OpenAI(temperature = 0.5)

tool = [search_tool]

agent = initialize_agent(tool, llm, agent="zero-shot-react-description", verbose=True)

response = agent.run("What are the latest top ranking football player in 2025?")

print(response)



> Entering new AgentExecutor chain...
 I should search for a website that has rankings for football players
Action: DuckDuckGo Search
Action Input: 'top ranking football players 2025'
Observation: As such, we've assembled a team of RotoWire's fantasy football experts to weigh in with their individual player rankings at the four primary positions: Quarterback, Running Back, Wide Receiver and Tight End. Here is how each of our experts ranked their top-10 QBs and TEs, as well as their top-15 RBs and WRs for the 2025 fantasy football season ... Here are the top 30 fantasy football players for 2025. 2025 Pro Bowl snubs:Which standout players got passed over in roster picks? ... Fantasy football: 2025 player rankings 1. WR Ja'Marr Chase ... Fantasy Football Top 25 2025 Rankings: Brian Thomas Jr. and Malik Nabers will be in high demand next season ... Player or players who just missed the cut: This is the range for the top two ... That said, when Higgins suited up in 2024, he consistently p

# Wikipedia Search

In [9]:
from langchain_community.utilities.wikipedia import WikipediaAPIWrapper

wiki_search_tool = Tool(
    name = "Wikipedia Search",
    func= WikipediaAPIWrapper().run,
    description= "Use this tool to search Wikipedia for information on any topic."
)

llm = OpenAI(temperature = 0.5)

tool = [wiki_search_tool]

agent = initialize_agent(tool, llm, agent="zero-shot-react-description", verbose=True)

response = agent.run("Who is Trump? Tell me the latest news of him")

print(response)



> Entering new AgentExecutor chain...
 I should try searching on Wikipedia
Action: Wikipedia Search
Action Input: Trump

c:\Users\rjcha\.conda\envs\agentdemo\Lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file c:\Users\rjcha\.conda\envs\agentdemo\Lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')



Observation: Page: Trumpism
Summary: Trumpism is a political movement in the United States that comprises the political ideologies associated with U.S. President Donald Trump and his political base. It incorporates ideologies such as right-wing populism, right-wing antiglobalism, national conservatism, and neo-nationalism, and features significant illiberal and authoritarian beliefs. Trumpists and Trumpians are terms that refer to individuals exhibiting its characteristics. There is significant academic debate over the prevalence of neo-fascist elements of Trumpism.
Trumpism is associated with the belief that the President is above the rule of law. It has been referred to as an American political variant of the far-right and the national-populist and neo-nationalist sentiment seen in multiple nations starting in the mid-late 2010s. Trump's political base has been compared to a cult of personality. Trump supporters became the largest faction of the United States Republican Party, with 

# SQLite database

In [10]:
import sqlite3

# Connect to SQLite database (creates example.db if it doesn't exist)
connection = sqlite3.connect("example_new.db")
cursor = connection.cursor()

# Create the inventory table
cursor.execute("""
CREATE TABLE IF NOT EXISTS inventory (
    id INTEGER PRIMARY KEY,
    product_name TEXT NOT NULL,
    stock_quantity INTEGER NOT NULL
)
""")

# Insert sample data
cursor.executemany("""
INSERT INTO inventory (product_name, stock_quantity) VALUES (?, ?)
""", [
    ("Product A", 150),
    ("Product B", 200),
    ("Product C", 300),
    ("Product D", 100),
    ("Product E", 50)
])

# Commit and close connection
connection.commit()
connection.close()

print("Database created and populated with sample data.")


Database created and populated with sample data.


In [12]:
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit 

In [13]:
# Step 1: Connect to the Database
db = SQLDatabase.from_uri("sqlite:///example_new.db")

# Step 3: Initialize the LLM
llm = OpenAI(temperature=0.5)

# Step 2: Create the SQL Database Toolkit

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

tools = toolkit.get_tools()

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent="zero-shot-react-description",
    verbose=True
)


query1 = "How many units of Product A are in stock?"
response1 = agent.run(query1)
print("Response:", response1)



> Entering new AgentExecutor chain...
 I need to check the database for the stock of Product A.
Action: sql_db_query
Action Input: SELECT units FROM products WHERE name = 'Product A'
Observation: Error: (sqlite3.OperationalError) no such table: products
[SQL: SELECT units FROM products WHERE name = 'Product A']
(Background on this error at: https://sqlalche.me/e/20/e3q8)
Thought: I need to use sql_db_list_tables to see which tables are available.
Action: sql_db_list_tables
Action Input: ""
Observation: inventory
Thought: I need to use sql_db_schema to see the schema and sample rows for the inventory table.
Action: sql_db_schema
Action Input: inventory
Observation: 
CREATE TABLE inventory (
	id INTEGER, 
	product_name TEXT NOT NULL, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (id)
)

/*
3 rows from inventory table:
id	product_name	stock_quantity
1	Product A	150
2	Product B	200
3	Product C	300
*/
Thought: Now that I know the structure of the inventory table, I can use sql_db_query

In [26]:
pip install arxiv

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6060 sha256=b5a5d86c86ab2ffaee762032d256e01814451d51592beca966e6cf6cf95930b4
  Stored in directory: c:\users\rjcha\appdata\local\pip\cache\wheels\3b\25\2a\105d6a15df6914f4d15047691c6c28f9052cc1173e40285d03
Successfully built sgmllib3k
Note: you may need to restart the kernel to use updated packages.


# ArxivQueryRun

In [27]:
from langchain.agents import initialize_agent, Tool
from langchain.llms import OpenAI
from langchain_community.tools.arxiv.tool import ArxivQueryRun

In [28]:
arxiv_tool = Tool(
    name="Arxiv Search",
    func=ArxivQueryRun().run,
    description="Use this tool to search academic papers on arXiv. Provide topics, authors, or keywords."
)

llm = OpenAI(temperature=0.5)

tools = [arxiv_tool]

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent="zero-shot-react-description",
    verbose=True
)


query = "Search for research papers on Object Detection in 2024."
response = agent.run(query)
print("Response from Agent:")
print(response)



> Entering new AgentExecutor chain...
 I should use the Arxiv Search tool to search for research papers.
Action: Arxiv Search
Action Input: 'Object Detection 2024'
Observation: Published: 2024-07-08
Title: Towards Reflected Object Detection: A Benchmark
Authors: Zhongtian Wang, You Wu, Hui Zhou, Shuiwang Li
Summary: Object detection has greatly improved over the past decade thanks to advances
in deep learning and large-scale datasets. However, detecting objects reflected
in surfaces remains an underexplored area. Reflective surfaces are ubiquitous
in daily life, appearing in homes, offices, public spaces, and natural
environments. Accurate detection and interpretation of reflected objects are
essential for various applications. This paper addresses this gap by
introducing a extensive benchmark specifically designed for Reflected Object
Detection. Our Reflected Object Detection Dataset (RODD) features a diverse
collection of images showcasing reflected objects in various contexts,
pro

In [ ]:
File Agents